In [1]:
import sys,os
__script_path=os.path.abspath(globals().get('__file__','.'))
__script_dir = os.path.dirname(__script_path)
root_dir = os.path.abspath(f'{__script_dir}/../..')
print(root_dir)
for lib in [root_dir][::-1]:
    if lib in sys.path:
        sys.path.remove(lib)
    sys.path.insert(0,lib)

from libs.common import *
from config.config import *

c:\Users\Admin\Data\ads_fraud_detection


c:/Users/Admin/Data/ads_fraud_detection


# 3. Tiền xử lí dữ liệu

In [2]:
save_dir=f"{exps_dir}/exp_prj_fake_jobs_postings"
if os.path.exists(save_dir) == False:
  os.makedirs(save_dir, exist_ok=True)

test_size=0.33
seed=42

In [3]:
test_size=0.33
seed=42
data=pd.read_csv(f'{save_dir}/df_EDA.csv')
data

,job_id,has_company_logo,fraudulent,combined_text
0,9116,1,1,"Mobile Developer-Back End US, CA, San Francis..."
1,2653,1,1,"Administrative Assistant US, NC, Charlotte 300..."
2,938,1,1,"Health & Environmental Professional US, CA, Ba..."
3,9972,1,1,Director of Engineering HMA Security Products ...
4,5442,1,1,"Easy Money US, FL, Deltona DMT Instant Rewar..."
...,...,...,...,...
27869,17873,1,0,"Product Manager US, CA, San Francisco Flite ..."
27870,17874,1,0,"Recruiting Coordinator US, NC, Charlotte R..."
27871,17876,1,0,"Account Director - Distribution CA, ON, Toron..."
27872,17877,1,0,"Payroll Accountant US, PA, Philadelphia WebL..."


In [4]:
data_train, data_test = train_test_split(data, test_size = 0.33, random_state=seed)
print(data_train.shape,data_test.shape)
data_train.to_csv(f'{save_dir}/datatrain.csv', index=None)
data_test.to_csv(f'{save_dir}/datatest.csv', index=None)

(18675, 4) (9199, 4)


In [5]:
data_train

,job_id,has_company_logo,fraudulent,combined_text
15602,2153,1,0,"Sr.Java Developer jobs in Maryland US, MD, Lan..."
7872,9969,1,1,"Communications Manager US, NY, NY Aptitude ..."
22942,11425,1,0,"Data Operations Specialist US, CA, San Francis..."
24950,13909,1,0,"Senior Technical Project Manager GB, CMD, Lond..."
14931,1245,1,0,"Senior Visual Designer GB, LND, London We’..."
...,...,...,...,...
21575,9871,1,0,"Content Marketing Manager PL, , We are Netg..."
5390,8248,1,1,"Payroll Clerk US, MD, ANNAPOLIS At DynCorp I..."
860,10113,1,1,"Subsea Process Plant Layout Designer US, TX, H..."
15795,2405,1,0,"Inside Sales Representative (TELECOMMUTE) US, ..."


In [13]:
# nltk.download('popular')
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('stopwords')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [7]:
import nltk

def check_resource(resource_name):
    try:
        nltk.data.find(resource_name)
        print(f"{resource_name} is already downloaded.")
    except LookupError:
        print(f"{resource_name} not found. Downloading now...")
        nltk.download(resource_name.split('/')[-1])

check_resource('corpora/stopwords.zip')
check_resource('tokenizers/punkt.zip')


corpora/stopwords.zip not found. Downloading now...
tokenizers/punkt.zip not found. Downloading now...


[nltk_data] Error loading stopwords.zip: Package 'stopwords.zip' not
[nltk_data]     found in index
[nltk_data] Error loading punkt.zip: Package 'punkt.zip' not found in
[nltk_data]     index


In [11]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'<.*?>', '', text)
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'[^a-zA-Z0-9]', ' ', text)
    tokens = nltk.word_tokenize(text)
    stop_words = set(nltk.corpus.stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    cleaned_text = ' '.join(tokens)
    
    return cleaned_text

In [14]:
try:
    data['combined_text'] = data['combined_text'].apply(clean_text)
except LookupError as e:
    print(f"Missing resource: {e}")
    raise


In [15]:
data

,job_id,has_company_logo,fraudulent,combined_text
0,9116,1,1,mobile developer back end us ca san francisco ...
1,2653,1,1,administrative assistant us nc charlotte 30000...
2,938,1,1,health environmental professional us ca bakers...
3,9972,1,1,director engineering hma security products us ...
4,5442,1,1,easy money us fl deltona dmt instant reward hi...
...,...,...,...,...
27869,17873,1,0,product manager us ca san francisco flite deli...
27870,17874,1,0,recruiting coordinator us nc charlotte respons...
27871,17876,1,0,account director distribution ca toronto vend ...
27872,17877,1,0,payroll accountant us pa philadelphia weblinc ...


In [16]:
data.isnull().sum()

job_id              0
has_company_logo    0
fraudulent          0
combined_text       0
dtype: int64

# 4.Chuẩn hóa dữ liệu

In [17]:
numeric_columns=dict(np.load(f'{save_dir}/columns_dtype.npz',allow_pickle=True))['numeric_columns']
numeric_columns

array(['job_id'], dtype='<U6')

In [18]:
category_columns=dict(np.load(f'{save_dir}/columns_dtype.npz',allow_pickle=True))['category_columns']
category_columns

array(['has_company_logo', 'fraudulent', 'combined_text'], dtype='<U16')

In [20]:
print(spacy.util.get_package_path('en_core_web_sm'))


c:\Users\Admin\Data\ads_fraud_detection\.conda\lib\site-packages\en_core_web_sm


In [16]:
nlp = spacy.load("en_core_web_sm")

def normalize_text(text):
    # Tokenize the text and apply lemmatization
    doc = nlp(text)
    normalized_words = [token.lemma_ for token in doc]
    normalized_text = ' '.join(normalized_words)
    return normalized_text


In [17]:
data['combined_text']=data['combined_text'].apply(normalize_text)

### Bert word embedding

In [21]:
text_columns = ['combined_text']

try:
    data['pos_features'] = data[text_columns].apply(lambda x: nltk.pos_tag(word_tokenize(x[0])), axis=1)
    data['pos_features'] = data['pos_features'].apply(lambda tags: ' '.join(tag[1] for tag in tags))
    # data['combined_text'] = data['combined_text'].apply(clean_text)
except LookupError as e:
    print(f"Missing resource: {e}")
    raise

X = data.drop('fraudulent', axis=1)
y = data['fraudulent']

vectorizer = CountVectorizer(ngram_range=(1, 2))
text_matrix_train = vectorizer.fit_transform(X['combined_text'])
pos_matrix_train = vectorizer.transform(X['pos_features'])
combined_matrix_train = hstack([text_matrix_train, pos_matrix_train])

with open(f'{save_dir}/countvectorizer.pkl', 'wb') as file:
    pickle.dump(vectorizer, file)

In [22]:
data

,job_id,has_company_logo,fraudulent,combined_text,pos_features
0,9116,1,1,mobile developer back end us ca san francisco ...,JJ NN RB VBP PRP MD VB JJ NN NN IN VBG NNS VBD...
1,2653,1,1,administrative assistant us nc charlotte 30000...,JJ NN PRP VBP JJ CD CD JJ NN VBN JJ NN NN CD N...
2,938,1,1,health environmental professional us ca bakers...,NN JJ JJ PRP MD VB CD CD NN VBG NNS VBD VBG NN...
3,9972,1,1,director engineering hma security products us ...,NN NN JJ NN NNS PRP MD VB JJ JJ NN VBG NNS VBD...
4,5442,1,1,easy money us fl deltona dmt instant reward hi...,JJ NN PRP VBP JJ NN JJ NN VBG NN JJ JJ NN NN N...
...,...,...,...,...,...
27869,17873,1,0,product manager us ca san francisco flite deli...,NN NN PRP MD VB JJ JJ NNS NN NN JJ NN NN NNS V...
27870,17874,1,0,recruiting coordinator us nc charlotte respons...,VBG NN PRP VBP JJ NNS VBP VBG NN VBZ VBG JJ JJ...
27871,17876,1,0,account director distribution ca toronto vend ...,NN NN NN MD VB NN VBG JJ JJ NN VBN VBP PRP VBG...
27872,17877,1,0,payroll accountant us pa philadelphia weblinc ...,NN VBP PRP JJ NN NN NN NN NN NNS NN JJS VBG JJ...


In [23]:
X=combined_matrix_train
y=data['fraudulent'].values

In [30]:
# # X_dense = X.toarray()

# X = pd.DataFrame(X_dense)
y = pd.DataFrame(y, columns=['fraudulent'])


In [31]:
from scipy import sparse

# Giả sử X là ma trận thưa
sparse.save_npz(f'{save_dir}/X.npz', X)
y.to_csv(f'{save_dir}/y.csv', index=False)


In [27]:
X

<27874x1859086 sparse matrix of type '<class 'numpy.int64'>'
	with 13266528 stored elements in Compressed Sparse Row format>

In [28]:
sparse.load_npz(f'{save_dir}/sparse_matrix.npz')

<27874x1859086 sparse matrix of type '<class 'numpy.int64'>'
	with 13266528 stored elements in Compressed Sparse Row format>

In [32]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [33]:
# x_train=data_train.drop('fraudulent',axis=1)
# y_train=data_train['fraudulent']
# x_train.to_csv(f'{save_dir}/x_train.csv', index=False)
sparse.save_npz(f'{save_dir}/x_train.npz', x_train)

y_train.to_csv(f'{save_dir}/y_train.csv', index=False)

In [35]:

np.unique(y_train)

array([0, 1], dtype=int64)

In [39]:
y_train=y_train.fraudulent

In [40]:
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weights_dict = {i : class_weights[i] for i in range(len(class_weights))}
np.savez(f'{save_dir}/class_weights_dict.npz', class_weights_dict = class_weights_dict)
class_weights_dict

{0: 0.9991486692355945, 1: 1.0008527827648115}

In [41]:
from imblearn.over_sampling import SMOTE
x_train, y_train = SMOTE().fit_resample(x_train, y_train)
sparse.save_npz(f'{save_dir}/x_train.npz', x_train)
y_train.to_csv(f'{save_dir}/y_train.csv', index=False)

In [42]:
object_cols_test=dict(np.load(f'{save_dir}/columns_dtype.npz',allow_pickle=True))['category_columns']
numeric_cols_test=dict(np.load(f'{save_dir}/columns_dtype.npz',allow_pickle=True))['numeric_columns']

# label_encoders = {}
# for column in object_cols_test:
#     label_encoder = LabelEncoder()
#     data_test[column] = label_encoder.fit_transform(data_test[column])
#     label_encoders[column] = label_encoder  # Lưu trữ label encoder nếu cần sau này

# scaler = StandardScaler()
# data_test[numeric_cols_test] = scaler.fit_transform(data_test[numeric_cols_test])

# x_test=data_test.drop('fraudulent',axis=1)
# y_test=data_test['fraudulent']
sparse.save_npz(f'{save_dir}/x_test.npz', x_test)

y_test.to_csv(f'{save_dir}/y_test.csv', index=False)